In [ ]:
import spacy
nlp = spacy.load("en_core_web_trf")


In [ ]:
from tensorflow.python.client import device_lib
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "CPU"

# this is to check if the gpu is present
print(device_lib.list_local_devices())


In [ ]:
import pickle
import numpy as np
from tqdm import tqdm
from glob import glob
import os


In [3]:
IMAGE_PATH = "./images/"
filename_list = []
for img in tqdm(glob(IMAGE_PATH + "*.jpg")):
    filename_list.append(os.path.basename(img))


hello


100%|██████████████████████████████| 195379/195379 [00:00<00:00, 1834887.09it/s]


In [4]:
# extract captions for images
CAPTIONS_PATH = "./captions/"

captions = []
filename_list = []
for file in tqdm(glob(CAPTIONS_PATH + "*.txt")):
    filename_list.append(os.path.basename(file))
    textfile = open(file, "r", encoding="utf8")
    captions.append(textfile.read())
    textfile.close()

# sanity check
print(len(filename_list), len(captions))


100%|█████████████████████████████████| 195379/195379 [00:21<00:00, 8958.80it/s]

195379 195379


In [8]:
captions


["Grab your Ilse boots, looks like we're back to rainy days ☔️ #Sydney #rainboots #ilsejacobsen #streetstyle #ootd #ootdwomen #luxury #danishbrand #blogger",
 "SPARKLE STACKERS || Another birthday done and dusted! Now it's time to stop gallivanting around and get back to work! For now at least.....😜 #100ofone #showmeyourrings #stackingrings #ringstack #silverring",
 'Clean up your act with our new maker in store SCENTSORIE Soaps. Using highest quality organic and non-organic essential oils, mineral rich clays, exfoliants, nourishing oils, activated charcoal and more.. #soap #soapmaker #dodarlo #eastsidesydney #oxfordstreet #darlinghurst #surryhills #sydney #sydneymakers #natural #organic #skincare #workshopmakery',
 "Success doesn't just come and find You. You have to go out and get it ❗️🔥🏀 #noexcusesperformance #australia #sydney #basketball #ballislife #aussiehoops#memories #handle #basketballgame #gametime #basketballtrainer #possibletraining #passion#focus #gametime #grind#fitnessa

In [5]:
import re

REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|,;:\\+-_%$]")
HASHTAG_RE = re.compile("#\S+")
TAG_RE = re.compile("@[a-z0-9_]+")
HTTP_RE = re.compile("http([^ ]*)")
WEBSITE_RE = re.compile("(www.)?[a-z0-9]+[.](com|net|org)")
BAD_SYMBOLS_RE = re.compile("[#\"\']")
PERIOD_RE = re.compile("[.]")
EXCLAMATION_RE = re.compile("[!]")
QUESTION_RE = re.compile("[?]")
ACCEPTED_CHARACTERS_RE = re.compile("[^a-zA-Z!.? ]")
SPACED_CHARACTERS_RE = re.compile("(?<=\s)[b-hj-z]\s")


def prep_caption(string):
    string = string.lower()
    string = re.sub(TAG_RE, "", string)
    # remove all hashtags from captions
    string = re.sub(HASHTAG_RE, "", string)
    string = re.sub(HTTP_RE, "", string)
    string = re.sub(WEBSITE_RE, "", string)
    string = re.sub(REPLACE_BY_SPACE_RE, " ", string)  # remove all
    string = re.sub(BAD_SYMBOLS_RE, "", string)
    # punctuation
    string = re.sub(PERIOD_RE, " PERIOD ", string)
    string = re.sub(EXCLAMATION_RE, " EXCLAMATION ", string)
    string = re.sub(QUESTION_RE, " QUESTIONMARK ", string)
    string = re.sub(ACCEPTED_CHARACTERS_RE, "", string)
    # fix multiple spaces in a row
    string = re.sub(" +", " ", string)
    # delete strings with spaced characters like : H E L L O
    if re.search(SPACED_CHARACTERS_RE, string) is not None:
        string = ""
    if string:
        if string[0] == " ":
            string = string[1:]
        if string[-1] == " ":
            string = string[:-1]
    return string


In [7]:
examples = captions[0:100]
print("EX -> prep")
for thing in examples:
    print("BEFORE")
    print(thing)
    print("AFTER")
    print(prep_caption(thing))
    print("\n")


EX -> prep
BEFORE
Grab your Ilse boots, looks like we're back to rainy days ☔️ #Sydney #rainboots #ilsejacobsen #streetstyle #ootd #ootdwomen #luxury #danishbrand #blogger
AFTER
grab your ilse boots looks like were back to rainy days


BEFORE
SPARKLE STACKERS || Another birthday done and dusted! Now it's time to stop gallivanting around and get back to work! For now at least.....😜 #100ofone #showmeyourrings #stackingrings #ringstack #silverring
AFTER
sparkle stackers another birthday done and dusted EXCLAMATION now its time to stop gallivanting around and get back to work EXCLAMATION for now at least


BEFORE
Clean up your act with our new maker in store SCENTSORIE Soaps. Using highest quality organic and non-organic essential oils, mineral rich clays, exfoliants, nourishing oils, activated charcoal and more.. #soap #soapmaker #dodarlo #eastsidesydney #oxfordstreet #darlinghurst #surryhills #sydney #sydneymakers #natural #organic #skincare #workshopmakery
AFTER
clean up your act with o

In [8]:
for i, text in tqdm(enumerate(captions)):
    captions[i] = prep_caption(text)


195379it [00:04, 39123.62it/s]


In [9]:
captions


['grab your ilse boots looks like were back to rainy days',
 'sparkle stackers another birthday done and dusted EXCLAMATION now its time to stop gallivanting around and get back to work EXCLAMATION for now at least',
 'clean up your act with our new maker in store scentsorie soaps using highest quality organic and non organic essential oils mineral rich clays exfoliants nourishing oils activated charcoal and more',
 'success doesnt just come and find you you have to go out and get it',
 'who could ever tire of this view',
 'sleeps to go til the weekend',
 'after all this rain we are ready to roll',
 'derma stamp bleed to heal many different skin concerns such as scarring pigmentation uneven skintone finelines wrinkles dark circles enlarged pores blemishes etc',
 'am so proud of this guy my bro riding to raise funds for childhood cancer for those of you who can support him charity with au',
 'i want a ticket to anywhere',
 'working on a customers bike from home today i couldnt resist pu

In [38]:
objs_to_del = np.where(np.array(captions) == "")[0]
print(len(objs_to_del))
for obj in objs_to_del.tolist():
    os.remove("./images/" + filename_list[obj][:-4] + ".jpg")
    os.remove("./captions/" + filename_list[obj])
filename_list = np.delete(np.array(filename_list), objs_to_del).tolist()
captions = np.delete(np.array(captions), objs_to_del).tolist()


49887


In [10]:
count = 0
for caption in captions:
    if not caption:
        count += 1
print(len(captions) - count)
print(count)


195379
0


In [15]:
# remove strings that are not in english
from langdetect import detect


In [16]:
examples = captions[:1000]


In [17]:
for ex in examples:
    if detect(ex) != "en":
        print(detect(ex) + "\n" + ex)


it
laboratorio milano earrings new collection laboratoriomilano
en
laboratorio milano earrings new collection laboratoriomilano
en
great moments of cinema topkapi jules dassin
en
so it definitely feels like winter is getting close who else is feeling like a holiday
en
count your blessing not your problem
af
somewhere in between night and day
fr
you have sides ! choose your ! pymble station
en
dont put your foot in your mind


In [62]:
count = 0
for caption in tqdm(captions):
    if detect(caption) != "en":
        count += 1
print(len(captions))
print(count)


100%|██████████████████████████████████| 404344/404344 [19:02<00:00, 353.85it/s]

404344
33421


In [64]:
non_english_idxs = []
for idx, caption in tqdm(enumerate(captions)):
    if detect(caption) != "en":
        non_english_idxs.append(idx)
for idx in non_english_idxs:
    captions[idx] = ""


404344it [20:44, 325.02it/s]


In [76]:
more_objs_to_del = np.where(np.array(captions) == '')[0]
for obj in more_objs_to_del.tolist():
    os.remove("./images/" + filename_list[obj][:-4] + ".jpg")
    os.remove("./captions/" + filename_list[obj])
filename_list = np.delete(np.array(filename_list), more_objs_to_del).tolist()
captions = np.delete(np.array(captions), more_objs_to_del).tolist()


In [80]:
print(len(filename_list))
print(len(filename_list) == len(captions))


370892
True


In [16]:
count = 0
for caption in tqdm(captions):
    doc = nlp(caption)
    if doc.ents:
        count += 1
print(len(captions))
print(count)


100%|██████████████████████████████████| 370892/370892 [23:03<00:00, 268.07it/s]

370892
209729


In [18]:
count = 0
for caption in captions[:1000]:
    doc = nlp(caption)
    for ent in doc.ents:
        print(ent.text, ent.label_)


annual DATE
ebc olympics EVENT
sydney GPE
sydneys beaches LOC
sydney GPE
last weekend DATE
two CARDINAL
sydney aquarium FAC
earlier today TIME
vegas GPE
this week DATE
tomorrow DATE
wednesday DATE
easter EVENT
sydney opera house FAC
europe LOC
tania PERSON
sydney youth orchestra ORG
sunday april DATE
verbrugghen hall FAC
frida PERSON
australia GPE
pm to pm TIME
today DATE
three CARDINAL
this week DATE
easter EVENT
american NORP
bewitched WORK_OF_ART
elizabeth montgomery PERSON
lizzie borden PERSON
sarah schmidt PERSON
paula hawkins PERSON
johnny ambon PERSON
tara iv FAC
two CARDINAL
last saturday DATE
bakers delight wynyard ORG
this friday march DATE
first ORDINAL
the sydney korean festival EVENT
tumbalong park FAC
korean NORP
april DATE
darling harbour LOC
asia LOC
hubbie PERSON
australian NORP
chelsea PERSON
north head manly GPE
night TIME
friday night TIME
today DATE
one CARDINAL
last year DATE
autumn day DATE
sydney GPE
today DATE
agung sedayu group part ORG
martinick PERSON
kotai 

davis PERSON
evans street LOC
warren PERSON
straya PERSON
sydney town hall FAC
sydney GPE
amsterdam GPE
week DATE
thursday DATE
last weeks DATE
bondi GPE
axel ORG
saturday th april DATE
mudgee region LOC
lola biggins PERSON
the white spotty heeler PERSON
brazilian NORP
brazilian NORP
fri PERSON
sat nd bday PERSON
take me back WORK_OF_ART
sydney GPE
parsley bay LOC
april DATE
pm TIME
kirkley street FAC
bowenfels GPE
minutes TIME
the to minutes to hours TIME
sydney harbour bridge FAC
australia GPE
one CARDINAL
marianne PERSON
saturday DATE
saturday nd april DATE
winter DATE
one CARDINAL
sydney GPE
punjabi NORP
first ORDINAL
biarritz ORG
sydney GPE
nsw GPE
australia GPE
march DATE
hilton sydney ORG
ronald PERSON
today DATE
sicilian NORP
cronulla GPE
first ORDINAL
tomorrow DATE
el minara ORG
rd ORDINAL
the sydney summer crit series EVENT
first ORDINAL
first ORDINAL
coogee GPE
th march pm DATE
malo ORG
one CARDINAL
saturday sunday DATE
arthur PERSON
the beautiful sister of the bride PERSON


In [25]:
doc = nlp(captions[5])
for ent in doc.ents:
    print(ent.text, ent.label_, type(ent.label_))


annual DATE <class 'str'>
ebc olympics EVENT <class 'str'>


In [31]:
count = 0
all_ents = {}
for i, caption in tqdm(enumerate(captions)):
    doc = nlp(caption)
    if doc.ents:
        count += 1
    all_ents[i] = doc.ents
print(len(captions))
print(count)


370892it [2:42:55, 37.94it/s]

370892
173682


In [36]:
ent_counter = Counter()
for i in range(len(captions)):
    for ent in all_ents[i]:
        ent_counter[ent.label_] += 1
print(ent_counter)


Counter({'DATE': 67426, 'GPE': 61285, 'PERSON': 48887, 'ORG': 21091, 'CARDINAL': 15713, 'FAC': 15360, 'TIME': 14714, 'NORP': 9475, 'LOC': 9432, 'ORDINAL': 9183, 'WORK_OF_ART': 5393, 'EVENT': 4935, 'PRODUCT': 3597, 'QUANTITY': 372, 'LANGUAGE': 318, 'MONEY': 171, 'PERCENT': 12, 'LAW': 12})


In [43]:
count = 0
entity_inds = []
for i in range(len(captions)):
    if all_ents[i]:
        count += 1
        entity_inds.append(i)
print(count)
print(len(entity_inds))
print(len(captions))
print(len(captions) - len(entity_inds))

for obj in entity_inds:
    os.remove("./images/" + filename_list[obj][:-4] + ".jpg")
    os.remove("./captions/" + filename_list[obj])
filename_list = np.delete(np.array(filename_list),
                          np.array(entity_inds)).tolist()
captions = np.delete(np.array(captions), np.array(entity_inds)).tolist()
print(len(filename_list))
print(len(captions))


173682
173682
370892
197210
197210
197210


In [47]:
count = 0
for caption in tqdm(captions):
    if "sydney" in caption:
        count += 1
print(len(captions))
print(count)


100%|██████████████████████████████| 197210/197210 [00:00<00:00, 4042058.13it/s]

197210
983


In [45]:
def split_sentence(sentence):
    return list(re.split("\W+", sentence))


In [46]:
for caption in captions[:100]:
    print(split_sentence(caption))


['stunning', 'ceremony', 'dress', 'in', 'champagne', 'perfect', 'for', 'that', 'special', 'occasion', 'sizes', 'to']
['grab', 'your', 'ilse', 'boots', 'looks', 'like', 'were', 'back', 'to', 'rainy', 'days']
['sparkle', 'stackers', 'another', 'birthday', 'done', 'and', 'dusted', 'now', 'its', 'time', 'to', 'stop', 'gallivanting', 'around', 'and', 'get', 'back', 'to', 'work', 'for', 'now', 'at', 'least']
['clean', 'up', 'your', 'act', 'with', 'our', 'new', 'maker', 'in', 'store', 'scentsorie', 'soaps', 'using', 'highest', 'quality', 'organic', 'and', 'non', 'organic', 'essential', 'oils', 'mineral', 'rich', 'clays', 'exfoliants', 'nourishing', 'oils', 'activated', 'charcoal', 'and', 'more']
['success', 'doesnt', 'just', 'come', 'and', 'find', 'you', 'you', 'have', 'to', 'go', 'out', 'and', 'get', 'it']
['who', 'could', 'ever', 'tire', 'of', 'this', 'view']
['sleeps', 'to', 'go', 'til', 'the', 'weekend']
['after', 'all', 'this', 'rain', 'we', 'are', 'ready', 'to', 'roll']
['thou', 'shalt'

In [49]:
split_captions = [split_sentence(caption) for caption in captions]


In [50]:
too_small_or_big_captions_indices = [ind for ind in range(len(split_captions)) if len(
    split_captions[ind]) <= 5 or len(split_captions[ind]) >= 50]


In [51]:
len(too_small_or_big_captions_indices)


1831

In [54]:
for obj in too_small_or_big_captions_indices:
    os.remove("./images/" + filename_list[obj][:-4] + ".jpg")
    os.remove("./captions/" + filename_list[obj])
filename_list = np.delete(np.array(filename_list), np.array(
    too_small_or_big_captions_indices)).tolist()
captions = np.delete(np.array(captions), np.array(
    too_small_or_big_captions_indices)).tolist()


In [55]:
len(filename_list)


195379

In [56]:
captions


['stunning ceremony dress in champagne perfect for that special occasion sizes to',
 'grab your ilse boots looks like were back to rainy days',
 'sparkle stackers another birthday done and dusted! now its time to stop gallivanting around and get back to work! for now at least',
 'clean up your act with our new maker in store scentsorie soaps using highest quality organic and non organic essential oils mineral rich clays exfoliants nourishing oils activated charcoal and more',
 'success doesnt just come and find you you have to go out and get it',
 'who could ever tire of this view',
 'sleeps to go til the weekend',
 'after all this rain we are ready to roll',
 'thou shalt not stress over the opinion of an irrelevant asshole! saved ya a plate!',
 'derma stamp bleed to heal many different skin concerns such as scarring pigmentation uneven skintone finelines wrinkles dark circles enlarged pores blemishes etc',
 'sydney fish market did not disappoint',
 'am so proud of this guy my bro ridi

In [19]:
from collections import Counter


In [22]:
# special tokens
PAD = "#PAD#"
UNK = "#UNK#"
START = "#START#"
END = "#END#"

# split sentence into tokens


def split_sentence(sentence):
    return list(re.split("\W+", sentence))


def generate_vocabulary(train_captions):
    """
    Return {token: index} for all train tokens (words) that occur 5 times or more, 
        `index` should be from 0 to N, where N is a number of unique tokens in the resulting dictionary.
    Use `split_sentence` function to split sentence into tokens.
    Also, add PAD (for batch padding), UNK (unknown, out of vocabulary), 
        START (start of sentence) and END (end of sentence) tokens into the vocabulary.
    """
    longest_list_ever = split_sentence(" ".join(train_captions))
    counter = Counter()
    total = 0
    for word in longest_list_ever:
        counter[word] += 1
        total += 1
    vocab = {word for word, count in counter.items() if count > 1}
    vocab = vocab.union({PAD, UNK, START, END})
    return {token: index for index, token in enumerate(sorted(vocab))}, counter, total


def caption_words_to_indices(train_captions, vocab):
    token_list = [[vocab[START]] + [vocab[word] if word in vocab else vocab[UNK]
                                    for word in split_sentence(caption)[:48]] + [vocab[END]] for caption in train_captions]
    return token_list


In [23]:
# prepare vocabulary
vocab, frequencies, word_count = generate_vocabulary(captions)
vocab_inverse = {idx: w for w, idx in vocab.items()}
print(len(vocab))


31956


In [24]:
print(len(frequencies))


62683


In [25]:
len(vocab)


31956

In [26]:
unk_count = 0
all_words_count = 0
for _, num in frequencies.items():
    if num < 2:
        unk_count += num
    all_words_count += num
print(unk_count)
print(all_words_count)
print(unk_count/all_words_count)


30731
2910121
0.010560042005126247


In [27]:
vocab_inverse = {idx: w for w, idx in vocab.items()}


In [31]:
with open("./other/vocab.pkl", "wb") as fp:
    pickle.dump(vocab, fp)


In [33]:
len(vocab)


31956

In [34]:
captions_indexed = caption_words_to_indices(captions, vocab)


In [36]:
for i, indexed_caption in tqdm(enumerate(captions_indexed)):
    np.save("./tokenized_caption/" +
            filename_list[i][:-4] + ".npy", np.array(indexed_caption))


195379it [01:39, 1959.77it/s]
